## Neural Collaborative Filtering Recommender

In [1]:
# 경고 무시
import warnings
warnings.filterwarnings('ignore')
# 데이터 처리 및 분석
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', None)
import numpy as np

# 머신러닝
import tensorflow as tf

# AWS 관련
import sagemaker
from sagemaker.tensorflow import TensorFlow
from sagemaker.utils import name_from_base
import boto3
import awswrangler as wr

# 시각화
import plotly.graph_objects as go

# 기타 유틸리티
import os
import json
import pickle
import kaggle
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/dante/Library/Application Support/sagemaker/config.yaml


True

In [2]:
os.makedirs('script', exist_ok=True)

In [3]:
%%writefile script/ncf.py
import tensorflow as tf
import argparse
import os
import numpy as np
import json

class DataLoader:
    @staticmethod
    def load_training_data(base_dir):
        """훈련 데이터를 로드하고 분할합니다."""
        # 'train.npy' 파일에서 훈련 데이터를 로드합니다.
        df_train = np.load(os.path.join(base_dir, 'train.npy'))
        # 파일 목록을 출력합니다.
        print("훈련 데이터 디렉토리 내용:")
        for file in os.listdir(base_dir):
            print(f"- {file}")
        # 데이터를 사용자, 아이템, 라벨로 분할합니다.
        print(df_train.shape)
        return np.split(np.transpose(df_train).flatten(), 3)

    @staticmethod
    def batch_generator(user_data, item_data, labels, batch_size, n_batch, shuffle, user_dim, item_dim):
        """훈련 및 테스트를 위한 배치를 생성합니다."""
        counter = 0
        training_index = np.arange(user_data.shape[0])

        if shuffle:
            # 학습 데이터를 무작위로 섞습니다.
            np.random.shuffle(training_index)

        while True:
            # 현재 배치의 인덱스를 선택합니다.
            batch_index = training_index[batch_size * counter:batch_size * (counter + 1)]
            # 사용자와 아이템 데이터를 원-핫 인코딩합니다.
            user_batch = tf.one_hot(user_data[batch_index], depth=user_dim)
            item_batch = tf.one_hot(item_data[batch_index], depth=item_dim)
            y_batch = labels[batch_index]
            counter += 1
            # 배치 데이터를 생성합니다.
            yield [user_batch, item_batch], y_batch

            if counter == n_batch:
                if shuffle:
                    # 모든 배치를 순회한 후 데이터를 다시 섞습니다.
                    np.random.shuffle(training_index)
                counter = 0

class NeuralCollaborativeFiltering:
    def __init__(self, user_dim, item_dim, dropout_rate=0.25):
        """Neural Collaborative Filtering 모델 초기화"""
        self.user_dim = user_dim
        self.item_dim = item_dim
        self.dropout_rate = dropout_rate

    def build_model(self):
        """Neural Collaborative Filtering 모델을 구축합니다."""
        # 사용자와 아이템 입력 레이어를 정의합니다.
        user_input = tf.keras.Input(shape=(self.user_dim,))
        item_input = tf.keras.Input(shape=(self.item_dim,))

        # 사용자와 아이템의 임베딩을 생성합니다.
        user_gmf_emb, user_mlp_emb = self._create_user_embeddings(user_input)
        item_gmf_emb, item_mlp_emb = self._create_item_embeddings(item_input)

        # GMF(General Matrix Factorization)와 MLP(Multi-Layer Perceptron) 출력을 계산합니다.
        gmf_output = self._general_matrix_factorization(user_gmf_emb, item_gmf_emb)
        mlp_output = self._multi_layer_perceptron(user_mlp_emb, item_mlp_emb)

        # 최종 출력을 계산합니다.
        output = self._neural_cf(gmf_output, mlp_output)

        # 모델을 생성하고 반환합니다.
        return tf.keras.Model(inputs=[user_input, item_input], outputs=output)

    def _create_user_embeddings(self, inputs):
        """GMF와 MLP를 위한 사용자 임베딩을 생성합니다."""
        # GMF를 위한 사용자 임베딩
        gmf_emb = tf.keras.layers.Dense(32, activation='relu')(inputs)
        # MLP를 위한 사용자 임베딩
        mlp_emb = tf.keras.layers.Dense(32, activation='relu')(inputs)
        return gmf_emb, mlp_emb

    def _create_item_embeddings(self, inputs):
        """GMF와 MLP를 위한 아이템 임베딩을 생성합니다."""
        # GMF를 위한 아이템 임베딩
        gmf_emb = tf.keras.layers.Dense(32, activation='relu')(inputs)
        # MLP를 위한 아이템 임베딩
        mlp_emb = tf.keras.layers.Dense(32, activation='relu')(inputs)
        return gmf_emb, mlp_emb

    def _general_matrix_factorization(self, user_emb, item_emb):
        """General Matrix Factorization 브랜치를 구현합니다."""
        # 사용자와 아이템 임베딩의 요소별 곱을 계산합니다.
        return tf.keras.layers.Multiply()([user_emb, item_emb])

    def _multi_layer_perceptron(self, user_emb, item_emb):
        """Multi-Layer Perceptron 브랜치를 구현합니다."""
        # 사용자와 아이템 임베딩을 연결합니다.
        concat_layer = tf.keras.layers.Concatenate()([user_emb, item_emb])
        # 드롭아웃 레이어를 적용하여 과적합을 방지합니다.
        dropout = tf.keras.layers.Dropout(self.dropout_rate)(concat_layer)

        # 여러 개의 완전 연결 레이어를 추가합니다.
        dense1 = tf.keras.layers.Dense(64, activation='relu')(dropout)
        dense2 = tf.keras.layers.Dense(32, activation='relu')(dense1)
        dense3 = tf.keras.layers.Dense(16, activation='relu')(dense2)
        dense4 = tf.keras.layers.Dense(8, activation='relu')(dense3)

        return dense4

    def _neural_cf(self, gmf, mlp):
        """GMF와 MLP 출력을 결합합니다."""
        # GMF와 MLP 출력을 연결합니다.
        concat_layer = tf.keras.layers.Concatenate()([gmf, mlp])
        # 최종 출력 레이어(시그모이드 활성화 함수를 사용한 이진 분류)
        return tf.keras.layers.Dense(1, activation='sigmoid')(concat_layer)

def train_model(x_train, y_train, n_user, n_item, num_epoch, batch_size):
    """Neural Collaborative Filtering 모델을 훈련시킵니다."""
    # 전체 배치 수를 계산합니다.
    num_batch = np.ceil(x_train[0].shape[0] / batch_size)

    # 모델 인스턴스를 생성합니다.
    ncf = NeuralCollaborativeFiltering(n_user, n_item)
    model = ncf.build_model()

    # 옵티마이저를 설정합니다.
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    # 모델을 컴파일합니다.
    model.compile(optimizer=optimizer,
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=['accuracy'])

    # 데이터 로더 인스턴스를 생성합니다.
    data_loader = DataLoader()
    # 모델을 훈련시킵니다.
    model.fit(
        data_loader.batch_generator(
            x_train[0], x_train[1], y_train,
            batch_size=batch_size, n_batch=num_batch,
            shuffle=True, user_dim=n_user, item_dim=n_item),
        epochs=num_epoch,
        steps_per_epoch=num_batch,
        verbose=2
    )

    return model

def parse_args():
    """명령줄 인자를 파싱합니다."""
    parser = argparse.ArgumentParser(description="Neural Collaborative Filtering 모델 훈련")
    parser.add_argument('--model_dir', type=str, help="모델 출력 디렉토리")
    parser.add_argument('--sm-model-dir', type=str, default=os.environ.get('SM_MODEL_DIR'), help="SageMaker 모델 출력 디렉토리")
    parser.add_argument('--train', type=str, default=os.environ.get('SM_CHANNEL_TRAINING'), help="훈련 데이터 디렉토리")
    parser.add_argument('--hosts', type=json.loads, default=json.loads(os.environ.get('SM_HOSTS')), help="분산 훈련을 위한 호스트")
    parser.add_argument('--current-host', type=str, default=os.environ.get('SM_CURRENT_HOST'), help="분산 훈련에서 현재 호스트")
    parser.add_argument('--epochs', type=int, default=3, help="훈련 에포크 수")
    parser.add_argument('--batch_size', type=int, default=256, help="훈련 배치 크기")
    parser.add_argument('--n_user', type=int, required=True, help="사용자 수")
    parser.add_argument('--n_item', type=int, required=True, help="아이템 수")

    return parser.parse_args()

if __name__ == "__main__":
    # 명령줄 인자를 파싱합니다.
    args = parse_args()

    # 훈련 데이터를 로드합니다.
    data_loader = DataLoader()
    user_train, item_train, train_labels = data_loader.load_training_data(args.train)

    # 모델을 훈련시킵니다.
    ncf_model = train_model(
        x_train=[user_train, item_train],
        y_train=train_labels,
        n_user=args.n_user,
        n_item=args.n_item,
        num_epoch=args.epochs,
        batch_size=args.batch_size
    )

    # 모델을 저장합니다.
    if args.current_host == args.hosts[0]:
        ncf_model.save(os.path.join(args.sm_model_dir, '000000001'), 'neural_collaborative_filtering.h5')


Overwriting script/ncf.py


SageMaker 세션 및 역할 설정

In [2]:
boto3_session = boto3.Session(profile_name='awstutor')
sagemaker_session = sagemaker.Session(boto_session=boto3_session)
role = os.environ.get('SAGEMAKER_EXECUTION_ROLE_ARN')

### 데이터 준비

데이터 다운로드

In [ ]:
kaggle_dataset_name = 'CooperUnion/anime-recommendations-database'
kaggle.api.authenticate()
kaggle.api.dataset_download_files(kaggle_dataset_name, path='./dataset/anime', unzip=True)

Dataset URL: https://www.kaggle.com/datasets/CooperUnion/anime-recommendations-database


데이터 로드 및 확인

In [5]:
anime = pd.read_csv('./dataset/anime/anime.csv')
rating = pd.read_csv('./dataset/anime/rating.csv')

In [118]:
anime.shape, rating.shape

((12294, 7), (7813737, 3))

In [6]:
anime.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [7]:
rating.head()

,user_id,anime_id,rating
0,1,20,-1
1,1,24,-1
2,1,79,-1
3,1,226,-1
4,1,241,-1


In [101]:
rating.rating.value_counts()

rating
 8     1646019
-1     1476496
 7     1375287
 9     1254096
 10     955715
 6      637775
 5      282806
 4      104291
 3       41453
 2       23150
 1       16649
Name: count, dtype: int64

In [9]:
anime.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12232 non-null  object 
 3   type      12269 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


데이터 전처리

In [157]:
# 장르 정보가 없는 애니메이션에 대해 'No Genre' 값을 할당
anime.loc[anime.genre.isna(), 'genre'] = 'No Genre'

# 1. 중복된 데이터의 평점 평균 계산
rating_mean = rating.groupby(['user_id', 'anime_id'])['rating'].mean().reset_index()

# 2. 원본 데이터에서 중복 제거 (첫 번째 항목 유지)
rating_deduped = rating.drop_duplicates(subset=['user_id', 'anime_id'], keep='first')

# 3. 중복 제거된 데이터의 평점을 평균 평점으로 업데이트
rating_deduped = rating_deduped.merge(rating_mean, on=['user_id', 'anime_id'], suffixes=('_old', ''))

# 4. 기존 'rating_old' 열 삭제
df_rating = rating_deduped.drop(columns=['rating_old'])

print(f"중복 제거 후 데이터 수: {len(df_rating)}")

df_rating.shape

중복 제거 후 데이터 수: 7813730


(7813730, 3)

In [158]:
# 사용자별 애니메이션 평가 횟수가 100회 이상인 사용자와
# 애니메이션별 평가 횟수가 100회 이상인 애니메이션만 선택

# 1. 사용자별 애니메이션 평가 횟수가 100회 이상인 사용자 선택
df_rating_tmp = df_rating.merge(
    (df_rating.groupby('user_id').count().anime_id > 100).reset_index(), 
        on='user_id', how='inner', suffixes=(None, '_tmp')
)

# 애니메이션별 평가 횟수가 100회 이상인 애니메이션 선택
df_rating_tmp = df_rating_tmp.set_index('anime_id').merge(
    (df_rating.groupby('anime_id').count().user_id > 100).reset_index(),
        on='anime_id', how='inner', suffixes=(None, '_tmp')
)

In [161]:
# 사용자와 애니메이션 모두 100회 이상 평가된 데이터만 선택
df_rating_adj = df_rating_tmp[df_rating_tmp.apply(lambda x: x.user_id_tmp and x.anime_id_tmp, axis=1)].drop(columns=['user_id_tmp', 'anime_id_tmp'])

# 조정된 데이터셋의 크기 확인
df_rating_adj.shape

(5963423, 3)

데이터 셔플 및 분할

In [162]:
# 데이터프레임을 무작위로 섞습니다.
# frac=1은 전체 데이터를 사용한다는 의미입니다.
suffled_ratings_df = df_rating_adj.sample(frac=1)

In [163]:
# 각 사용자별로 처음 30개의 데이터를 테스트 세트로 분리
df_test = suffled_ratings_df.groupby(['user_id']).apply(lambda x: x.iloc[:30]).reset_index(drop=True)

# 각 사용자별로 30번째 이후의 데이터를 훈련 세트로 분리
df_train = suffled_ratings_df.groupby(['user_id']).apply(lambda x: x.iloc[30:]).reset_index(drop=True)

In [164]:
# 데이터 중복 또는 손실 여부 확인을 위한 무결성 검사
assert suffled_ratings_df.shape[0] == df_train.shape[0] + df_test.shape[0]

print("훈련 데이터 크기:", df_train.shape)
print("테스트 데이터 크기:", df_test.shape)

훈련 데이터 크기: (5223138, 3)
테스트 데이터 크기: (740285, 3)


In [165]:
# 훈련 데이터와 테스트 데이터를 CSV 파일로 저장
df_train.to_csv('./dataset/anime/user_anime_train.csv', index=False)
df_test.to_csv('./dataset/anime/user_anime_test.csv', index=False)

print("데이터 분할 및 저장 완료")


데이터 분할 및 저장 완료


고유한 아이템 및 사용자 수 계산

In [166]:
unique_items = suffled_ratings_df.anime_id.unique()
unique_users = suffled_ratings_df.user_id.unique()   

부정적 샘플 생성 및 병합
```
사용자가 어떤 작품에 대해 긍정적으로 평가했다면, 
그 데이터만으로는 모델 학습에 한계가 있습니다. 
그래서 각 사용자별로 아직 보지 않은 작품 중에서 임의로 몇 개를 골라 부정적인 샘플로 추가합니다. 
이렇게 하면 모델이 더 균형 잡힌 학습을 할 수 있죠.
```

In [167]:
def generate_negative_samples(user_ids, anime_ids, items, n_neg):
    """이 함수는 모든 긍정적 레이블에 대해 n_neg개의 부정적 레이블을 생성합니다.
    
    @param user_ids: 사용자 ID 목록
    @param anime_ids: 애니메이션 ID 목록
    @param items: 고유한 애니메이션 ID 목록
    @param n_neg: 샘플링할 부정적 레이블의 수
    
    @return df_neg: 부정적 샘플 데이터프레임
    
    """
    from tqdm import tqdm
    neg = []
    ui_pairs = zip(user_ids, anime_ids)
    records = set(ui_pairs)
    
    # 모든 긍정적 레이블 케이스에 대해
    for (u, i) in tqdm(records):
        # n_neg개의 부정적 레이블 생성
        for _ in range(n_neg):
            # 무작위로 샘플링된 애니메이션이 해당 사용자에게 존재하는 경우
            j = np.random.choice(items)
            while (u, j) in records:
                # 재샘플링
                j = np.random.choice(items)
            neg.append([u, j, 0])
    # 나중에 연결을 위해 pandas 데이터프레임으로 변환
    df_neg = pd.DataFrame(neg, columns=['user_id', 'anime_id', 'rating'])
    
    return df_neg

In [168]:
# 부정적 샘플 생성
neg_train = generate_negative_samples(
    user_ids=df_train.user_id.values, 
    anime_ids=df_train.anime_id.values,
    items=unique_items,
    n_neg=1
)

100%|██████████| 5223138/5223138 [00:35<00:00, 148892.76it/s]


In [169]:
neg_train.head(10)

,user_id,anime_id,rating
0,58046,1556,0
1,35486,1469,0
2,52481,25939,0
3,39522,1168,0
4,61254,6162,0
5,9192,8792,0
6,57892,1744,0
7,33085,6867,0
8,31200,529,0
9,52366,8728,0


In [170]:
print(f'created {neg_train.shape[0]:,} negative samples')

created 5,223,138 negative samples


In [171]:
# 훈련 데이터와 테스트 데이터에서 'user_id'와 'anime_id' 열만 선택하고 'rating' 열을 1로 설정
df_train = df_train[['user_id', 'anime_id']].assign(rating=1)
df_test = df_test[['user_id', 'anime_id']].assign(rating=1)

# 훈련 데이터에 부정적 샘플 추가
df_train = pd.concat([df_train, neg_train], ignore_index=True)

In [172]:
pickle.dump(unique_users, open('./dataset/anime/n_user.pkl', 'wb'))
pickle.dump(unique_items, open('./dataset/anime/n_item.pkl', 'wb'))
print("고유 사용자 수", len(unique_users))
print("고유 아이템 수", len(unique_items))

고유 사용자 수 24677
고유 아이템 수 4575


### 컨테이너 모델 준비

S3 데이터 경로 설정 및 업로드

In [6]:
bucket_name = 'dante-sagemaker'
project_name = 'ncf-recommender'
train_path = os.path.join(f's3://{bucket_name}/{project_name}/train/')
script_path = os.path.join(f's3://{bucket_name}/{project_name}/script/')
output_path = os.path.join(f's3://{bucket_name}/{project_name}/output/') 
checkpoint_path = os.path.join(f's3://{bucket_name}/{project_name}/checkpoints/')

In [174]:
# 데이터를 로컬에 저장
location = 'dataset/anime'
local_train_path = os.path.join(location, 'train.npy')
local_test_path = os.path.join(location, 'test.npy')

# 훈련 및 테스트 데이터를 NumPy 배열로 저장
np.save(local_train_path, df_train.values)
np.save(local_test_path, df_test.values)

# S3 버킷에 파일 업로드
# NCF 모델 스크립트 업로드
wr.s3.upload(local_file="script/ncf.py", path=os.path.join(script_path, 'ncf.py'), boto3_session=boto3_session)
# 훈련 데이터 업로드
wr.s3.upload(local_file=local_train_path, path=os.path.join(train_path, 'train.npy'), boto3_session=boto3_session)
# 테스트 데이터 업로드
wr.s3.upload(local_file=local_test_path, path=os.path.join(train_path, 'test.npy'), boto3_session=boto3_session)

2024-08-14 15:39:06,621 WARNING Connection pool is full, discarding connection: dante-sagemaker.s3.ap-northeast-2.amazonaws.com. Connection pool size: 10
2024-08-14 15:39:06,708 WARNING Connection pool is full, discarding connection: dante-sagemaker.s3.ap-northeast-2.amazonaws.com. Connection pool size: 10


모델 컨테이너 설정

In [9]:
ncf_estimator = TensorFlow(
    entry_point='script/ncf.py',
    role=role,
    sagemaker_session=sagemaker_session,
    output_path=output_path,
    train_instance_count=1,
    train_instance_type='ml.c5.2xlarge',
    framework_version='2.1.0',
    py_version='py3',
    distributions={
        'parameter_server': {'enabled': True}
    },
    hyperparameters= {
        'epochs': 1,
        'batch_size': 512, 
        'n_user': len(unique_users), 
        'n_item': len(unique_items)
    },
    use_spot_instances=True,
    max_wait=3600,
    max_run=3600,
    checkpoint_s3_uri=checkpoint_path,
)

distributions has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [11]:
from sagemaker.inputs import TrainingInput

training_job_name = name_from_base(f"{project_name}-training")

ncf_estimator.fit(
    inputs= {
        'training': TrainingInput(s3_data=train_path, content_type='application/x-npy'),
    },
    job_name=training_job_name
)

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: ncf-recommender-training-2024-08-14-08-03-00-881


2024-08-14 08:03:01 Starting - Starting the training job...
2024-08-14 08:03:17 Starting - Preparing the instances for training...
2024-08-14 08:03:57 Downloading - Downloading the training image...
2024-08-14 08:04:12 Training - Training image download completed. Training in progress.2024-08-14 08:04:14,508 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2024-08-14 08:04:14,515 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2024-08-14 08:04:14,692 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2024-08-14 08:04:14,706 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2024-08-14 08:04:14,719 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2024-08-14 08:04:14,728 sagemaker-containers INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {
        "sagemaker_parameter_server_enabled": true
    },
    "ch

### 모델 평가

엔드포인트 생성

In [14]:
# 훈련 작업 목록 조회
sagemaker_client = boto3_session.client('sagemaker')
response = sagemaker_client.list_training_jobs(
    StatusEquals='Completed',
    SortBy='CreationTime',
    SortOrder='Descending',
    MaxResults=5
)

job_names = []

print("최근 완료된 훈련 작업 목록:")
for job in response['TrainingJobSummaries']:
    if job['TrainingJobName'].startswith(f'{project_name}-training'):
        job_names.append(job['TrainingJobName'])
        print(f"작업 이름: {job['TrainingJobName']}")
        print(f"생성 시간: {job['CreationTime']}")
        print(f"완료 시간: {job['TrainingEndTime']}")
        print(f"상태: {job['TrainingJobStatus']}")
        print("-" * 50)
job_names

최근 완료된 훈련 작업 목록:
작업 이름: ncf-recommender-training-2024-08-14-08-03-00-881
생성 시간: 2024-08-14 17:03:01.211000+09:00
완료 시간: 2024-08-14 17:15:17.547000+09:00
상태: Completed
--------------------------------------------------


['ncf-recommender-training-2024-08-14-08-03-00-881']

In [15]:
# 최근 완료된 훈련작업을 통한 모델 아티팩트 위치 얻기
sagemaker_client = boto3_session.client('sagemaker')
response = sagemaker_client.describe_training_job(TrainingJobName=job_names[0])
model_artifacts = response['ModelArtifacts']['S3ModelArtifacts']

print(f"모델 아티팩트 위치: {model_artifacts}")

모델 아티팩트 위치: s3://dante-sagemaker/ncf-recommender/output/ncf-recommender-training-2024-08-14-08-03-00-881/output/model.tar.gz


In [20]:
from sagemaker.tensorflow import TensorFlowModel

ncf_estimator = TensorFlowModel(
    model_data=model_artifacts,
    role=role,
    sagemaker_session=sagemaker_session,
    framework_version='2.1.0',
    entry_point='script/ncf.py',
)

In [21]:
endpoint_name = name_from_base(project_name + '-ncf-endpoint')

ncf_estimator.deploy(
    initial_instance_count=1, 
    instance_type="ml.m5.2xlarge", 
    endpoint_name=endpoint_name,
)

INFO:sagemaker.tensorflow.model:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating model with name: tensorflow-inference-2024-08-14-08-19-10-784
INFO:sagemaker:Creating endpoint-config with name ncf-recommender-ncf-endpoint-2024-08-14-08-19-05-263
INFO:sagemaker:Creating endpoint with name ncf-recommender-ncf-endpoint-2024-08-14-08-19-05-263


----!

입력 데이터 준비

In [3]:
import numpy as np
import tensorflow as tf

In [4]:
n_user, n_item = 24677, 4575
project_name = 'ncf-recommender'

In [22]:
n_user, n_item = len(unique_users), len(unique_items)

print("number of unique users", n_user)
print("number of unique items", n_item)

number of unique users 24677
number of unique items 4575


In [5]:
# 테스트 데이터 로드
df_test = np.load('dataset/anime/test.npy')
# 원핫인코딩으로 인해 많은 메로리가 필요하므로, 5만행으로 데이터를 줄여 진행합니다.
sub_df_test = df_test[:50000]
user_test, item_test, y_test = np.split(np.transpose(sub_df_test).flatten(), 3)
user_test.shape, item_test.shape, y_test.shape

((50000,), (50000,), (50000,))

In [6]:
import gc
from tqdm import tqdm

In [7]:
# 모델 입력을 위해 테스트 데이터를 원-핫 인코딩
batch_size = 1000  # 메모리 사용량을 줄이기 위해 배치 크기 설정

test_user_data = []
test_item_data = []

for i in tqdm(range(0, len(user_test), batch_size)):
    batch_user = user_test[i:i+batch_size]
    batch_item = item_test[i:i+batch_size]
    
    user_batch = tf.one_hot(batch_user, depth=n_user).numpy()
    item_batch = tf.one_hot(batch_item, depth=n_item).numpy()
    
    test_user_data.extend(user_batch.tolist())
    test_item_data.extend(item_batch.tolist())

# 메모리 정리
del user_batch, item_batch
gc.collect()

  0%|          | 0/50 [00:00<?, ?it/s]

100%|██████████| 50/50 [00:31<00:00,  1.60it/s]


0

엔드포인트 실시간 추론

In [8]:
# 엔드포인트 목록 조회
sagemaker_client = boto3_session.client('sagemaker')
response = sagemaker_client.list_endpoints()
endpoint_name = None
# 엔드포인트 이름 출력
print("현재 존재하는 엔드포인트 목록:")
for endpoint in response['Endpoints']:
    if endpoint['EndpointName'].startswith(f'{project_name}-ncf-endpoint'):
        endpoint_name = endpoint['EndpointName']
        print(endpoint['EndpointName'])
        break


현재 존재하는 엔드포인트 목록:
ncf-recommender-ncf-endpoint-2024-08-14-08-19-05-263


In [9]:
# 배치 예측 수행
runtime = boto3_session.client('runtime.sagemaker')
batch_size = 100
y_pred = []
for idx in tqdm(range(0, len(test_user_data), batch_size)):
    # 테스트 샘플을 TensorFlow Serving이 수용 가능한 형식으로 재구성
    input_vals = {
     "instances": [
         {'input_1': u, 'input_2': i} 
         for (u, i) in zip(test_user_data[idx:idx+batch_size], test_item_data[idx:idx+batch_size])
    ]}
    # 엔드포인트 호출
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType='application/json',
        Body=json.dumps(input_vals)
    )
    # 응답 파싱
    pred = json.loads(response['Body'].read().decode())
 
    # 예측 결과 저장
    y_pred.extend([i[0] for i in pred['predictions']])

100%|██████████| 1/1 [00:02<00:00,  2.52s/it]


In [17]:
# 예측 확률을 선호도로 간주하여 상위 추천 애니메이션 확인
print('1. 예측 데이터 5개')
print('-'*80)
print(y_pred[:5], end='\n\n\n')

# 사용자-아이템 쌍 데이터프레임 생성, 예측값은 확률 그대로 사용
pred_df = pd.DataFrame({
    '사용자ID': user_test,
    '애니메이션ID': item_test,
    '선호도': y_pred
})

print('2. 사용자-아이템 쌍 데이터프레임')
print('-'*80)
print(pred_df.head(), end='\n\n\n')

# 사용자별 상위 5개 추천 애니메이션 목록
print('3. 사용자별 상위 5개 추천 애니메이션 목록')
print('-'*80)
top_recommendations = pred_df.groupby('사용자ID').apply(lambda x: x.nlargest(5, '선호도')['애니메이션ID'].tolist()).head()
print(top_recommendations.to_frame(), end='\n\n\n')

# 5명의 사용자에 대한 상세 추천 정보 출력
print('4. 5명의 사용자에 대한 상세 추천 정보')
print('-'*80)
for user_id in top_recommendations.index[:5]:
    user_recs = pred_df[pred_df['사용자ID'] == user_id].nlargest(5, '선호도')
    print(f"사용자 ID: {user_id}")
    print(user_recs[['애니메이션ID', '선호도']])
    print()

1. 예측 데이터 5개
--------------------------------------------------------------------------------
[0.573741, 0.573741, 0.573741, 0.573741, 0.573741]


2. 사용자-아이템 쌍 데이터프레임
--------------------------------------------------------------------------------
   사용자ID  애니메이션ID       선호도
0      1    10578  0.573741
1      1    16011  0.573741
2      1     9330  0.573741
3      1    10079  0.573741
4      1    15451  0.573741


3. 사용자별 상위 5개 추천 애니메이션 목록
--------------------------------------------------------------------------------
                                      0
사용자ID                                  
1            [356, 2787, 79, 24, 10578]
5          [4214, 1887, 150, 4472, 918]
7      [552, 4472, 31704, 15583, 18671]
11           [44, 3652, 532, 1709, 104]


4. 5명의 사용자에 대한 상세 추천 정보
--------------------------------------------------------------------------------
사용자 ID: 1
    애니메이션ID       선호도
24      356  0.938439
13     2787  0.884002
6        79  0.815785
12       24  0.788223
0     10

엔드포인트 삭제

In [18]:
sm = boto3_session.client('sagemaker')
sm.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': '51c5350c-6e30-46d8-929c-c5508680f10f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '51c5350c-6e30-46d8-929c-c5508680f10f',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Wed, 14 Aug 2024 08:44:06 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}